In [1]:
from math import log

Nếu mã hóa mã Hamming bằng cách nhân vector với ma trận, phép toán này sẽ có độ phức tạp là $O(m^2)$ với $m$ là độ dài chuỗi.<br/>
Ta có thể mã hóa một cách tối ưu hơn từ quan sát sau: Ma trận sinh của mã tuyến tính chứa ma trận đơn vị, tức là các vị trí trong chuỗi ban đầu không bị thay đổi giá trị, việc mã hóa có thể thực hiện bằng cách thêm $O(log m)$ giá trị vào chuỗi gốc (với mã Hamming), cách làm này sẽ có độ phức tạp là $O(m log m)$ <br/>
Để đơn giản, ta sẽ dùng ma trận kiểm tra $H$ với cột thứ $i$ (chỉ số bắt đầu từ 1) là biểu diễn nhị phân của $i$ (việc này thỏa mãn tính chất của mã Hamming). Các cột chỉ gồm 1 số 1, hay nói cách khác có vị trí $2^n$ sẽ ứng với các bit được thêm vào. Minh họa với mã Hamming(7,4)<br/>

| p_1 | p_2 | d_1 | p_4 | d_2 | d_3 | d_4 |
| :-: | :-: | :-: | :-: | :-: | :-: | :-: |
| 1 | 0 | 1 | 0 | 1 | 0 | 1 |
| 0 | 1 | 1 | 0 | 0 | 1 | 1 |
| 0 | 0 | 0 | 1 | 1 | 1 | 1 |

Từ tính chất của ma trận kiểm tra, ta có thể tính bit được thêm ở vị trí $2^n$ từ các bit ở vị trí $i$ sao có biểu diễn nhị phân của $i$ bằng 1 tại vị trí thứ $n$ thông qua phép toán OR.

In [2]:
def number_of_parity(data):
    #calculate the number of parity bit one has to add
    length=len(data)
    for r in range(int(log(length)),length):
        if 2**r >= length+r+1:
            return r
def add_bit(data, r):
    #padding original data with r redundant bit
    for i in range(r):
        data.insert(2**i - 1, 0)
    return data
def encoding(data):
    #calculate parity bits
    r = number_of_parity(data)
    data = add_bit(data, r)
    length = len(data)
    for i in range(r):
        p = 0
        for j in range(1, length + 1):
            if (j & 2**i) and (j != 2**i):
                p ^= data[j - 1]
        data[2**i - 1] = p
    return data, r

Ta giải mã bằng cách nhân vector nhận được với ma trận $H$. Do ta đã sử dụng ma trận $H$ có cột thứ $i$ là biểu diễn nhị phân của $i$ để mã hóa, ở bước mã hóa ta sẽ sử dụng ma trận này. Vị trí thứ $i$ của vector đầu ra được tính bằng cách thực hiện phép toán OR cho vị trí $i$ của các cột với vị trí có biểu diễn nhị phân bằng 1 tại $i$, ví dụ vị trí thứ 1 được tính qua các cột 1, 3, 5, 7,... , vị trí thứ 2 được tính qua các cột 2, 3, 6, 7,...

In [3]:
def decoding(data, r):
    length = len(data)
    out=[]
    for i in range(r):
        p = 0
        for j in range(1, length + 1):
            if (j & 2**i):
                p ^= data[j - 1]
        out.append(p)
    pos = sum([2**j * i for j, i in enumerate(out)])
    data[pos-1] ^= 1
    return data, pos

In [4]:
input = [1,1,0,0,1,0,1]
data, r = encoding(input)
print(data)

[0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1]


In [5]:
import random
def noise(data):
    corrupted_pos = random.randint(0, len(data))
    data[corrupted_pos] ^= 1
    return data

In [6]:
corrupted_data = noise(data)
print(corrupted_data)

[0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1]


In [7]:
correct = decoding(corrupted_data, r)
correct

([0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1], 4)

In [9]:
def back(data, r):
    for i in range(r):
        data[2**i - 1] = 0
    print(data)